installing

set up

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import logging
from pathlib import Path
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
import os


logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

HOME_DIR = Path("/home/jovyan/workspace/Lingua-Emoca/")
os.chdir(HOME_DIR)

SAVE_DIR = HOME_DIR / 'scripts' / 'results' / 'multimodal_opensmile_bert' / 'experiments' / 'lstm_max_pool_pad'
SAVE_DIR.mkdir(parents=True, exist_ok=True)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
NUM_EPOCHS = 200
BATCH_SIZE = 16  # Reduced for stability
LEARNING_RATE = 0.001  # Increased for better convergence
WEIGHT_DECAY = 0.01  # Reduced

# --- SEED GENERATION ---
# Generate 10 truly random seeds for the experiment
experiment_seeds = np.random.randint(0, 10000, size=10)
logger.info(f"Starting experiment with seeds: {experiment_seeds}")

def set_seed(seed):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

MLP

In [ ]:
class FocalLoss(nn.Module):
    """Focal Loss implementation for addressing class imbalance"""
    def __init__(self, alpha=1.0, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

class SimpleMLP(nn.Module):
    def __init__(self, input_size, hidden_sizes, num_classes, dropout_rate=0.3):
        super(SimpleMLP, self).__init__()
        
        layers = []
        current_size = input_size
        
        for hidden_size in hidden_sizes:
            layers.extend([
                nn.Linear(current_size, hidden_size),
                nn.ReLU(),
                nn.BatchNorm1d(hidden_size),
                nn.Dropout(dropout_rate)
            ])
            current_size = hidden_size
        
        layers.append(nn.Linear(current_size, num_classes))
        self.network = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.network(x)
        


running experiments

In [ ]:
def run_experiment(X_data, y_labels):
    all_acc = []
    all_f1 = []
    
    for run_idx, seed in enumerate(experiment_seeds):
        logger.info(f"Run {run_idx+1}/10 | Seed: {seed}")
        set_seed(seed)
        
        # 1. Model Initialization
        # input_size is the number of openSMILE features
        model = SimpleMLP(input_size=X_data.shape[1], hidden_sizes=[256, 128], num_classes=len(np.unique(y_labels))).to(device)
        
        # 2. Optimizer with WEIGHT_DECAY (L2 Regularization)
        optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
        criterion = FocalLoss().to(device)
        
        # [Insert DataLoading & Training Loop here...]
        # After training, evaluate:
        
        model.eval()
        # ... (Get predictions for test set) ...
        
        acc = accuracy_score(y_true, y_pred)
        f1 = f1_score(y_true, y_pred, average='weighted')
        
        all_acc.append(acc)
        all_f1.append(f1)
        
        logger.info(f"Seed {seed} Result -> Acc: {acc:.4f}, F1: {f1:.4f}")

    # Final Statistics
    logger.info("="*30)
    logger.info(f"Final Accuracy: {np.mean(all_acc):.4f} +/- {np.std(all_acc):.4f}")
    logger.info(f"Final F1-Score: {np.mean(all_f1):.4f} +/- {np.std(all_f1):.4f}")
    logger.info("="*30)